## Problem Statement

Welcome to DCH Designs. We help you find your perfect Fixer Upper, and take you through the design process that adds value to your future home in Ames, IA.

First we will predict values for a home we will first import our data and clean it up before making a model to predict houses.

Then we will go through and find you a home, give you designs, and the new estimated value of the home.

## Background

Ames, Iowa is home to famed Iwoa State University.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')

## Checking imported csv files

**Training Shape:** 2051, 81

**Training Columns:** See [Data Dictionary]('./code/data_dictionary.md')

Id | PID | MS SubClass | MS Zoning | Lot Frontage | Lot Area | Street | Alley | Lot Shape | Land Contour | Utilities | Lot Config | Land Slope | Neighborhood | Condition 1 | Condition 2 | Bldg Type | House Style | Overall Qual | Overall Cond | Year Built | Year Remod/Add | Roof Style | Roof Matl | Exterior 1st | Exterior 2nd | Mas Vnr Type | Mas Vnr Area | Exter Qual | Exter Cond | Foundation | Bsmt Qual | Bsmt Cond | Bsmt Exposure | BsmtFin Type 1 | BsmtFin SF 1 | BsmtFin Type 2 | BsmtFin SF 2 | Bsmt Unf SF | Total Bsmt SF | Heating | Heating QC | Central Air | Electrical | 1st Flr SF | 2nd Flr SF | Low Qual Fin SF | Gr Liv Area | Bsmt Full Bath | Bsmt Half Bath | Full Bath | Half Bath | Bedroom AbvGr | Kitchen AbvGr | Kitchen Qual | TotRms AbvGrd | Functional | Fireplaces | Fireplace Qu | Garage Type | Garage Yr Blt | Garage Finish | Garage Cars | Garage Area | Garage Qual | Garage Cond | Paved Drive | Wood Deck SF | Open Porch SF | Enclosed Porch | 3Ssn Porch | Screen Porch | Pool Area | Pool QC | Fence | Misc Feature | Misc Val | Mo Sold | Yr Sold | Sale Type | SalePrice


MS Subclass would need to be one hot encoded if used.

MS Zoning would too... This is a column to quickly explore because most residences should be Zoned for residential. The main differences will be to see if higher density versus lower and medium density residential zoning has affects the price. This would need to be one-hot encoded and I might one-hot encode as different residential categories and 'other' depending on what a quick analysis finds.

Lot Area should have a decently high correlation with price.

Street and Alley would need to be one-hot encoded. 

Overall Quality and Overall Condition seem to be easy ordinal categories to see if there is a correlation.

Sale condition would have an obvious affect on prices as short sales, foreclosures, off normal market sales between family members would most likely have lower prices than a typical sale.




In [4]:
df_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

## Handling Nulls and Cleaning up Columns

There are a lot of columns and a lot of nulls in the set. List of columns with Null values are: **'Lot Frontage',
 'Alley',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin SF 1',
 'BsmtFin Type 2',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Cars',
 'Garage Area',
 'Garage Qual',
 'Garage Cond',
 'Pool QC',
 'Fence',
 'Misc Feature'**



**Lot Frontage** has 330 null values. I will need to extrapolate values for these nulls if I decide to use **Lot Frontage** as a feature to make predictions.

**Alley** has 1911 null values.

**Mas Vnr Type** and **Mas Vnr Area** both have 22 nulls. I assume they do not have masonry veneers on the exterior. I will explore this further before deciding whether or not to use these columns as features.

**Bas 

**Fireplace Quality** Homes without a basement would not have a quality. Categorey most likely will not be used as a feature.

In [5]:
#I used https://stackoverflow.com/questions/53137100/filter-pandas-dataframe-columns-with-null-data
# to create a list of columns that have nulls.

df_train.columns[df_train.isnull().any()].tolist()

['Lot Frontage',
 'Alley',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin SF 1',
 'BsmtFin Type 2',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Cars',
 'Garage Area',
 'Garage Qual',
 'Garage Cond',
 'Pool QC',
 'Fence',
 'Misc Feature']

### Lot Frontage

In [6]:
df_train[df_train['Lot Frontage'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
7,145,535154050,20,RL,NaN,12160,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1959,1959,Hip,CompShg,Plywood,Plywood,BrkFace,180.0,TA,TA,CBlock,TA,TA,No,Rec,1000.0,Unf,0.0,188.0,1188.0,GasA,Fa,Y,SBrkr,1188,0,0,1188,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1959.0,RFn,2.0,531.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,5,2010,COD,142000
8,1942,535353130,20,RL,NaN,15783,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1Story,5,5,1952,1952,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,292.0,Unf,0.0,632.0,924.0,GasA,TA,Y,SBrkr,924,0,0,924,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1952.0,Unf,1.0,420.0,TA,TA,Y,0,324,0,0,0,0,NaN,MnPrv,Shed,400,6,2007,WD,112500
23,12,527165230,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,6,7,1992,2007,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,Gd,PConc,Gd,TA,No,ALQ,935.0,Unf,0.0,233.0,1168.0,GasA,Ex,Y,SBrkr,1187,0,0,1187,1.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Attchd,1992.0,Fin,2.0,420.0,TA,TA,Y,483,21,0,0,0,0,NaN,GdPrv,Shed,500,3,2010,WD,185000
27,1534,909277040,50,RL,NaN,11700,Pave,Grvl,IR1,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,1Fam,1.5Fin,5,6,1937,1995,Gable,CompShg,WdShing,Wd Shng,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,606.0,Unf,0.0,336.0,942.0,GasA,Ex,Y,SBrkr,1265,673,0,1938,0.0,0.0,2,0,4,1,Gd,7,Min2,1,Gd,Detchd,1937.0,Unf,1.0,240.0,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,198000
31,2860,909276010,70,RL,NaN,11435,Pave,NaN,IR1,HLS,AllPub,Corner,Mod,Crawfor,Norm,Norm,1Fam,2Story,8,7,1929,1950,Gable,CompShg,BrkFace,Stucco,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,792.0,792.0,GasA,Fa,Y,SBrkr,792,725,0,1517,0.0,0.0,1,0,3,1,Gd,7,Typ,2,Gd,Detchd,1931.0,Unf,2.0,400.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,230000
42,983,923275040,85,RL,NaN,9101,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,6,1978,1978,Gable,CompShg,Plywood,Plywood,BrkFace,104.0,TA,Gd,PConc,Gd,TA,Av,GLQ,1097.0,Unf,0.0,0.0,1097.0,GasA,Ex,Y,SBrkr,1110,0,0,1110,1.0,0.0,1,0,1,1,Gd,4,Typ,1,TA,Attchd,1978.0,Fin,2.0,602.0,TA,TA,Y,303,30,0,0,0,0,NaN,NaN,NaN,0,7,2009,WD,165500
50,2511,533221080,160,FV,NaN,2998,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,TwnhsE,2Story,6,5,2000,2000,Gable,CompShg,MetalSd,MetalSd,BrkFace,513.0,Gd,TA,PConc,Gd,TA,No,GLQ,353.0,Unf,0.0,403.0,756.0,GasA,Ex,Y,SBrkr,768,756,0,1524,0.0,0.0,2,1,2,1,Gd,4,Typ,0,NaN,Detchd,2000.0,Unf,2.0,440.0,TA,TA,Y,0,32,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,166000
59,997,527107010,60,RL,NaN,15038,Pave,NaN,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1996,1996,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,PConc,Gd,TA,No,Rec,138.0,Unf,0.0,778.0,916.0,GasA,Gd,Y,SBrkr,916,720,0,1636,0.0,0.0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1996.0,Fin,2.

In [7]:
df_train['Lot Shape'].value_counts()

Reg    1295
IR1     692
IR2      55
IR3       9
Name: Lot Shape, dtype: int64

In [8]:
df_train.groupby('Lot Shape')['Lot Frontage', 'Lot Area'].agg(['count', 'mean', 'median'])

<ipython-input-8-3128da55a1b7>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_train.groupby('Lot Shape')['Lot Frontage', 'Lot Area'].agg(['count', 'mean', 'median'])


Lot Frontage                    Lot Area                       
                 count        mean median    count          mean   median
Lot Shape                                                                
IR1                469   74.582090   73.0      692  11470.709538  10456.5
IR2                 36   59.416667   51.0       55  19995.454545  13682.0
IR3                  6  115.666667   78.0        9  21102.222222  15295.0
Reg               1210   66.968595   65.0     1295   8815.707336   8773.0

I am checking below if there is correlation between Lot Shape and Lot Frontage, or if it primarily depends on the Lot Area.

In [9]:
df_train_lotshape = pd.get_dummies(data = df_train, columns = ['Lot Shape'])

In [10]:
df_train_lotshape.corr()['Lot Frontage']

Id                -0.024967
PID               -0.091291
MS SubClass       -0.389790
Lot Frontage       1.000000
Lot Area           0.574024
Overall Qual       0.194808
Overall Cond      -0.054455
Year Built         0.109504
Year Remod/Add     0.085052
Mas Vnr Area       0.218218
BsmtFin SF 1       0.214178
BsmtFin SF 2       0.036740
Bsmt Unf SF        0.127605
Total Bsmt SF      0.356584
1st Flr SF         0.463900
2nd Flr SF         0.013700
Low Qual Fin SF   -0.000300
Gr Liv Area        0.383856
Bsmt Full Bath     0.111441
Bsmt Half Bath    -0.030858
Full Bath          0.152812
Half Bath          0.035842
Bedroom AbvGr      0.220414
Kitchen AbvGr      0.015856
TotRms AbvGrd      0.362704
Fireplaces         0.253456
Garage Yr Blt      0.067865
Garage Cars        0.297143
Garage Area        0.358457
Wood Deck SF       0.115180
Open Porch SF      0.184958
Enclosed Porch    -0.003302
3Ssn Porch         0.031855
Screen Porch       0.074819
Pool Area          0.156575
Misc Val           0

In [11]:
df_train['lot_area_frontage'] = df_train['Lot Frontage'] / df_train['Lot Area']

In [12]:
l_a_f_mean = df_train['Lot Area'] * df_train['lot_area_frontage'].mean()

In [13]:
df_train['Lot Frontage'].fillna(l_a_f_mean, inplace = True)

In [14]:
df_train['Lot Frontage'].isnull().sum()

0

### Alley Column

In [15]:
df_train['Alley'].isnull().mean()

0.931740614334471

In [16]:
df_train['Alley'].value_counts()

Grvl    85
Pave    55
Name: Alley, dtype: int64

The Column 'Alley' has 85 homes with Gravel and 55 with Pavel alleys. The remainder I assume do not have alleys. Since 93% of homes do **not** have an alley I will probably not use the column as a feature. 

### Mas Vnr Type & Mas Ven Area

A majority of homes do not have masonry veneer, so I will fill null values with 'None.'

In [17]:
df_train['Mas Vnr Type'].value_counts()

None       1218
BrkFace     630
Stone       168
BrkCmn       13
Name: Mas Vnr Type, dtype: int64

In [18]:
df_train['Mas Vnr Type'].fillna('None', inplace = True)

In [19]:
df_train['Mas Vnr Area'].fillna(0, inplace = True)

### Bsmt Qual

In [20]:
df_train['Bsmt Qual'].value_counts()

TA    887
Gd    864
Ex    184
Fa     60
Po      1
Name: Bsmt Qual, dtype: int64

In [21]:
df_train['Bsmt Qual'].fillna('None', inplace = True)

In [22]:
qual_dict = {np.nan : 0, 'None' : 0, 'Po' : 1, 'Fa' : 2, 'Ta' : 3, 'Gd' : 4, 'TA' : 5}

In [23]:
df_train['Bsmt Qual'].isna().sum()

0

In [24]:
df_train['Bsmt Qual'] = df_train['Bsmt Qual'].map(qual_dict)

In [25]:
df_train['Bsmt Qual'].dtype

dtype('float64')

### Bsmt Cond

In [26]:
df_train['Bsmt Cond'].value_counts()

TA    1834
Gd      89
Fa      65
Po       5
Ex       3
Name: Bsmt Cond, dtype: int64

In [27]:
df_train['Bsmt Cond'].isna().sum()

55

In [28]:
df_train['Bsmt Cond'].fillna('None', inplace = True)

In [29]:
df_train['Bsmt Cond'] = df_train['Bsmt Cond'].map(qual_dict)

In [30]:
df_train['Bsmt Cond'].dtype

dtype('float64')

### Bsmt Exposure

In [31]:
df_train['Bsmt Exposure'].value_counts()

No    1339
Av     288
Gd     203
Mn     163
Name: Bsmt Exposure, dtype: int64

In [32]:
df_train['Bsmt Exposure'].isnull().sum()

58

In [33]:
df_train['Bsmt Exposure'].fillna('None', inplace = True)

In [34]:
bsmt_dict = {'No' : 0, 'None' : 0, 'Mn' : 1, 'Av' : 2, 'Gd' : 3}

In [35]:
df_train['Bsmt Exposure'] = df_train['Bsmt Exposure'].map(bsmt_dict)

### Bsmt Fin Type 1

In [44]:
df_train['BsmtFin Type 1'].value_counts()

GLQ    615
Unf    603
ALQ    293
BLQ    200
Rec    183
LwQ    102
Name: BsmtFin Type 1, dtype: int64

In [46]:
df_train['BsmtFin Type 1'].isnull().sum()

55

In [47]:
df_train['BsmtFin Type 1'].fillna('None', inplace = True)

In [48]:
bsmt_fin_dict = {'None' : 0,'Unf' : 0, 'LwQ' : 1, 'Rec' : 2, 'BLQ' : 3, 'ALQ' : 4, 'GLQ' : 5}

In [49]:
df_train['BsmtFin Type 1'] = df_train['BsmtFin Type 1'].map(bsmt_fin_dict)

### BsmtFin SF 1

In [51]:
df_train['BsmtFin SF 1'].value_counts()

0.0       657
24.0       17
16.0       10
300.0       7
384.0       7
20.0        7
700.0       6
624.0       6
456.0       6
360.0       6
312.0       6
368.0       6
1200.0      5
648.0       5
330.0       5
288.0       5
553.0       5
432.0       5
637.0       5
686.0       5
625.0       5
120.0       5
276.0       5
841.0       4
336.0       4
495.0       4
904.0       4
507.0       4
500.0       4
528.0       4
476.0       4
600.0       4
247.0       4
758.0       4
1218.0      4
319.0       4
784.0       4
28.0        4
252.0       4
851.0       4
732.0       4
450.0       4
410.0       4
468.0       4
425.0       4
547.0       4
338.0       4
836.0       4
350.0       4
549.0       4
375.0       4
104.0       4
56.0        4
588.0       4
626.0       4
510.0       4
544.0       4
492.0       3
915.0       3
560.0       3
602.0       3
776.0       3
594.0       3
353.0       3
437.0       3
299.0       3
240.0       3
527.0       3
644.0       3
786.0       3
280.0       3
532.0 

In [58]:
df_train['BsmtFin SF 1'].isnull().sum()

1

In [62]:
#https://datatofish.com/rows-with-nan-pandas-dataframe/
# to troubleshoot showing just the np.nan row

df_train[df_train['BsmtFin SF 1'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [63]:
df_train['BsmtFin SF 1'].fillna(0, inplace = True)

### BsmtFin Type 2

In [65]:
df_train['BsmtFin Type 2'].isnull().sum()

56

In [66]:
df_train['BsmtFin Type 2'].fillna('None', inplace = True)

In [67]:
df_train['BsmtFin Type 2'] = df_train['BsmtFin Type 2'].map(bsmt_fin_dict)

### BsmtFin SF 2

In [68]:
df_train[df_train['BsmtFin SF 2'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,0.0,0,NaN,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [69]:
df_train['BsmtFin SF 2'].fillna(0, inplace = True)

### Bsmt Unf SF

In [70]:
df_train[df_train['Bsmt Unf SF'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,0.0,0,0.0,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [73]:
df_train['Bsmt Unf SF'].fillna(0, inplace = True)

### Total Bsmt SF

In [72]:
df_train[df_train['Total Bsmt SF'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,0.0,0,0.0,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [74]:
df_train['Total Bsmt SF'].fillna(0, inplace = True)

### Bsmt Full Bath

In [75]:
df_train[df_train['Bsmt Full Bath'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
616,1498,908154080,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,7,1959,1996,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,Slab,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,GasA,TA,Y,SBrkr,3820,0,0,3820,NaN,NaN,3,1,5,1,Ex,11,Typ,2,Gd,Attchd,1959.0,Unf,2.0,624.0,TA,TA,Y,0,372,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,284700,0.002617
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [77]:
df_train['Bsmt Full Bath'].fillna(0, inplace = True)

### Bsmt Half Bath

In [78]:
df_train[df_train['Bsmt Half Bath'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,lot_area_frontage
616,1498,908154080,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,7,1959,1996,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,Slab,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,GasA,TA,Y,SBrkr,3820,0,0,3820,0.0,NaN,3,1,5,1,Ex,11,Typ,2,Gd,Attchd,1959.0,Unf,2.0,624.0,TA,TA,Y,0,372,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,284700,0.002617
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,GasA,TA,Y,FuseA,896,0,0,896,0.0,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000,0.016667


In [79]:
df_train['Bsmt Full Bath'].fillna(0, inplace = True)

### Pool QC Column

In [36]:
df_train['Pool QC'].value_counts()

Gd    4
Fa    2
TA    2
Ex    1
Name: Pool QC, dtype: int64

In [37]:
df_train['Pool QC'].isnull().mean()

0.9956118966357874

The Column 'Pool QC' has 9 total entries. The remainder null values I assume means they do not have pools. Since 99.5% of homes do **not** have a pool I will not use the column as a feature.

### Fence

In [38]:
df_train['Fence'].value_counts()

MnPrv    227
GdPrv     83
GdWo      80
MnWw      10
Name: Fence, dtype: int64

The column 'Fence' would have to be changed to values 1-5 with NaN changed to a value to 0 in order to represent no fence.

### Misc Feature

In [39]:
df_train['Misc Feature'].value_counts()

Shed    56
Gar2     4
Othr     3
Elev     1
TenC     1
Name: Misc Feature, dtype: int64

The column Misc Feature represents features such as a second garage, shed, or elevator. If this column is used for predicting, then all 0's after one-hot encoding would represent no Misc features.

## Changing Obj type to numericals

In order to perform analysis, the object type columns will need to be converted into numerical values. 

In [40]:
df_train.dtypes.value_counts()

object     39
int64      29
float64    14
dtype: int64

There are 42 columns that need to either be converted into numericals or dropped in order to be used in analysis.

## Potential Feature Columns:

In [41]:
[i for i in df_train.columns if 'Qual' in i]

['Overall Qual',
 'Exter Qual',
 'Bsmt Qual',
 'Low Qual Fin SF',
 'Kitchen Qual',
 'Garage Qual']

The columns: **'Overall Qual',
 'Exter Qual',
 'Bsmt Qual',
 'Low Qual Fin SF',
 'Kitchen Qual',
 'Garage Qual'** are ordinal but will have to be converted into numerical values.
 
 As my analysis focuses on finding a home, upgrading or updating, and then repricing; finding a correlation between these categories and the overall price is extremely important.

In [42]:
df_train['Yr Sold'].value_counts()

2007    498
2009    446
2006    438
2008    435
2010    234
Name: Yr Sold, dtype: int64

Something to keep in mind is the recession and real estate crash starting in 2008. I'd expect values to be higher in 2007 than the following years.